In [1]:
! pip install sqlalchemy

In [2]:
# This script uploads the validated MFA data from local machine to Snoflake database
import os
from sqlalchemy import create_engine, Column, Integer, String
from dotenv import load_dotenv
import snowflake.connector
import warnings

# Loading the environment variables from .env file
load_dotenv()

# Ignore warnings
warnings.filterwarnings("ignore")

# Snowflake engine for creating databases and warehouses
snowflake_engine = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
    )
)

try:
    # Snowflake operations
    snowflake_connection = snowflake_engine.connect()

    # Create databases
    create_raw_database_query = "CREATE OR REPLACE DATABASE RAW;"
    create_dbt_database_query = "CREATE OR REPLACE DATABASE DBT;"
    results = snowflake_connection.execute(create_raw_database_query)
    results = snowflake_connection.execute(create_dbt_database_query)

    # Use RAW database
    results = snowflake_connection.execute("USE DATABASE RAW")

    # Create warehouse
    create_warehouse_query = """CREATE OR REPLACE WAREHOUSE TRANSFORM
        WITH WAREHOUSE_SIZE='X-SMALL'
        AUTO_SUSPEND = 180
        AUTO_RESUME = TRUE
        INITIALLY_SUSPENDED=TRUE;"""
    results = snowflake_connection.execute(create_warehouse_query)

    # Create the RAW.MFA_TABLE
    create_raw_table_query = """CREATE OR REPLACE TABLE MFA_TABLE (
        Name_of_the_topic STRING,
        Year INTEGER,
        Level INTEGER,
        Introduction_Summary STRING,
        Learning_Outcomes STRING,
        Link_to_the_Summary_Page STRING(255),
        Link_to_the_PDF_file STRING(255)
    );"""
    results = snowflake_connection.execute(create_raw_table_query)

    # Create stage
    create_stage_query = """CREATE STAGE DUMMY DIRECTORY = (ENABLE = true)"""
    results = snowflake_connection.execute(create_stage_query)

    # Upload data to stage
    upload_to_stage_query = """PUT 'file://C:/Users/deepa/Documents/Repo/Assignment3/validated_CFA.csv' @DUMMY"""
    results = snowflake_connection.execute(upload_to_stage_query)

    # Copy data from stage to table
    copy_stage_to_table_raw_query = """COPY INTO MFA_TABLE
        FROM @DUMMY
        FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1)
        PATTERN = '.*validated_CFA.csv.gz'
        ON_ERROR = 'CONTINUE';"""
    results = snowflake_connection.execute(copy_stage_to_table_raw_query)

finally:
    print("Done")
    snowflake_connection.close()
    snowflake_engine.dispose()


Done
